### Import Required Libraries and Set Up Environment Variables

In [29]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [30]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [32]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL

query_url = (
    f"{url}"
    f"q=love&"  # General search term for keyword 'love'
    f"fq={filter_query}&"  # Filter query for section, type, and headline
    f"sort={sort}&"  # Sort by newest
    f"fl={field_list}&"  # Fields to return
    f"begin_date={begin_date}&"  # Start date for search
    f"end_date={end_date}&"  # End date for search
    f"api-key={nyt_api_key}"  # Your API key
)


In [33]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
for page in range(20):
    # create query with a page number
    # API results show 10 articles at a time
    query_url_with_page = f"{query_url}&page={page}"
    
    # Make a "GET" request and retrieve the JSON
    response = requests.get(query_url_with_page)
    
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    try:
        # loop through the reviews["response"]["docs"] and append each review to the list
        reviews = response.json()
        if 'response' in reviews and 'docs' in reviews['response']:
            for review in reviews['response']['docs']:
                reviews_list.append(review)

            # Print the page that was just retrieved
            print(f"Page {page} retrieved successfully.")
        else:

            # Print the page number that had no results then break from the loop
            print(f"Page {page} had no results.")
            break
    # Handle any errors
    except Exception as e:
        print(f"Error on page {page}: {str(e)}")
        break

Page 0 retrieved successfully.
Page 1 retrieved successfully.
Page 2 retrieved successfully.
Page 3 retrieved successfully.
Page 4 retrieved successfully.
Page 5 retrieved successfully.
Page 6 retrieved successfully.
Page 7 retrieved successfully.
Page 8 retrieved successfully.
Page 9 retrieved successfully.
Page 10 retrieved successfully.
Page 11 retrieved successfully.
Page 12 retrieved successfully.
Page 13 retrieved successfully.
Page 14 retrieved successfully.
Page 15 retrieved successfully.
Page 16 retrieved successfully.
Page 17 retrieved successfully.
Page 18 retrieved successfully.
Page 19 retrieved successfully.


In [41]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data

first_five_reviews = reviews_list[:5]

for i, review in enumerate(first_five_reviews):
    formatted_review = json.dumps(review, indent=4)
    print(f"Review {i+1}:")
    print(formatted_review)
    print("\n" + "="*80 + "\n")

Review 1:
{
    "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
    "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
    "source": "The New York Times",
    "headline": {
        "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
        "kicker": null,
        "content_kicker": null,
        "print_headline": "The Attachment Diaries",
        "name": null,
        "seo": null,
        "sub": null
    },
    "keywords": [
        {
            "name": "subject",
            "value": "Movies",
            "rank": 1,
            "major": "N"
        },
        {
            "name": "creative_works",
            "value": "The Attachment Diaries (Movie)",
            "rank": 2,
            "major": "N"
        },
        {
            "name": "persons",
            "value": "Diment, Valentin Javier",
            "rank": 3,
            "major": "N"
        }

In [71]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()

df_reviews = pd.json_normalize(reviews_list)
df_reviews

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,https://www.nytimes.com/2022/12/01/movies/lady...,The new Netflix adaptation of D.H. Lawrence’s ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2022-12-01T16:40:55+0000,768,‘Lady Chatterley’s Lover’ Review: When Connie ...,None,None,A Love Whose Name Is Often Spoken,None,None,None,By A.O. Scott,"[{'firstname': 'A.', 'middlename': 'O.', 'last...",None
216,https://www.nytimes.com/2022/11/23/movies/soun...,A young singer gets more than she bargained fo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2022-11-23T15:00:09+0000,277,‘The Sound of Christmas’ Review: A Gospel Sing...,None,None,The Sound Of Christmas,None,None,None,By Concepción de León,"[{'firstname': 'Concepción', 'middlename': Non...",None
217,https://www.nytimes.com/2022/11/17/movies/the-...,Elegance Bratton’s autobiographical first feat...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2022-11-17T12:00:07+0000,836,"‘The Inspection’ Review: Boot Camp, a Love Story",critic’s pick,None,"A Few Good Men, Some With Secrets",None,None,None,By A.O. Scott,"[{'firstname': 'A.', 'middlename': 'O.', 'last...",None
218,https://www.nytimes.com/2022/11/17/movies/bone...,Luca Guadagnino’s latest stars Timothée Chalam...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2022-11-17T12:00:06+0000,710,‘Bones and All’ Review: You Eat What You Are,None,None,"It’s Eat, Prey, Love on a Journey of Self-Disc...",None,None,None,By A.O. Scott,"[{'firstname': 'A.', 'middlename': 'O.', 'last...",None


In [73]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
df_reviews['title'] = df_reviews['headline.main'].apply(
    lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")+7]
)
df_reviews


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries’ Revie
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever’ Revie
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love’ Revie
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children’ Revie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,https://www.nytimes.com/2022/12/01/movies/lady...,The new Netflix adaptation of D.H. Lawrence’s ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2022-12-01T16:40:55+0000,768,‘Lady Chatterley’s Lover’ Review: When Connie ...,None,None,A Love Whose Name Is Often Spoken,None,None,None,By A.O. Scott,"[{'firstname': 'A.', 'middlename': 'O.', 'last...",None,Lady Chatterley’s Lover’ Revie
216,https://www.nytimes.com/2022/11/23/movies/soun...,A young singer gets more than she bargained fo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2022-11-23T15:00:09+0000,277,‘The Sound of Christmas’ Review: A Gospel Sing...,None,None,The Sound Of Christmas,None,None,None,By Concepción de León,"[{'firstname': 'Concepción', 'middlename': Non...",None,The Sound of Christmas’ Revie
217,https://www.nytimes.com/2022/11/17/movies/the-...,Elegance Bratton’s autobiographical first feat...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2022-11-17T12:00:07+0000,836,"‘The Inspection’ Review: Boot Camp, a Love Story",critic’s pick,None,"A Few Good Men, Some With Secrets",None,None,None,By A.O. Scott,"[{'firstname': 'A.', 'middlename': 'O.', 'last...",None,The Inspection’ Revie
218,https://www.nytimes.com/2022/11/17/movies/bone...,Luca Guadagnino’s latest stars Timothée Chalam...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2022-11-17T12:00:06+0000,710,‘Bones and All’ Review: You Eat What You Are,None,None,"It’s Eat, Prey, Love on a Journey of Self-Disc...",None,None,None,By A.O. Scott,"[{'firstname': 'A.', 'middlename': 'O.', 'last...",None,Bones and A

In [75]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
df_reviews['keywords'] = df_reviews['keywords'].apply(extract_keywords)
df_reviews

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries’ Revie
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever’ Revie
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love’ Revie
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children’ Revie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,https://www.nytimes.com/2022/12/01/movies/lady...,The new Netflix adaptation of D.H. Lawrence’s ...,The New York Times,subject: Movies;creative_works: Lady Chatterle...,2022-12-01T16:40:55+0000,768,‘Lady Chatterley’s Lover’ Review: When Connie ...,None,None,A Love Whose Name Is Often Spoken,None,None,None,By A.O. Scott,"[{'firstname': 'A.', 'middlename': 'O.', 'last...",None,Lady Chatterley’s Lover’ Revie
216,https://www.nytimes.com/2022/11/23/movies/soun...,A young singer gets more than she bargained fo...,The New York Times,subject: Movies;persons: Ne-Yo;creative_works:...,2022-11-23T15:00:09+0000,277,‘The Sound of Christmas’ Review: A Gospel Sing...,None,None,The Sound Of Christmas,None,None,None,By Concepción de León,"[{'firstname': 'Concepción', 'middlename': Non...",None,The Sound of Christmas’ Revie
217,https://www.nytimes.com/2022/11/17/movies/the-...,Elegance Bratton’s autobiographical first feat...,The New York Times,subject: Movies;creative_works: The Inspection...,2022-11-17T12:00:07+0000,836,"‘The Inspection’ Review: Boot Camp, a Love Story",critic’s pick,None,"A Few Good Men, Some With Secrets",None,None,None,By A.O. Scott,"[{'firstname': 'A.', 'middlename': 'O.', 'last...",None,The Inspection’ Revie
218,https://www.nytimes.com/2022/11/17/movies/bone...,Luca Guadagnino’s latest stars Timothée Chalam...,The New York Times,subject: Movies;creative_works: Bones and All ...,2022-11-17T12:00:06+0000,710,‘Bones and All’ Review: You Eat What You Are,None,None,"It’s Eat, Prey, Love on a Journey of Self-Disc...",None,None,None,By A.O. Scott,"[{'firstname': 'A.', 'middlename': 'O.', 'last...",None,Bones and All’ Revie


In [136]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database

df_reviews['title'] = df_reviews['headline.main'].apply(
    lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")]
)

titles = df_reviews['title'].to_list()
for title in titles:
    print(title)

The Attachment Diaries
What’s Love Got to Do With It?’ Probably a Lo
You Can Live Forever
A Tourist’s Guide to Love
Other People’s Children
One True Loves
The Lost Weekend: A Love Story
A Thousand and One
Your Place or Mine
Love in the Time of Fentanyl
Pamela, a Love Story
In From the Side
After Love
Alcarràs
Nelly & Nadine
Lady Chatterley’s Lover
The Sound of Christmas
The Inspection
Bones and All
My Policeman
About Fate
Waiting for Bojangles
I Love My Dad
A Love Song
Alone Together
Art of Love
The Wheel
Thor: Love and Thunder
Both Sides of the Blade
Fire of Love
Love & Gelato
Stay Prayed Up
Benediction
Dinner in America
In a New York Minute
Anaïs in Love
I Love America
See You Then
La Mami
Love After Love
Deep Water
Lucy and Desi
Cyrano
The In Between
Book of Love
Lingui, the Sacred Bonds
The Pink Cloud
A Journal for Jordan
West Side Story
Aulcie
Love Is Love Is Love
Love Hard
Bergman Island
Hard Luck Love Song
South of Heaven
Wife of a Spy
Happier Than Ever
Together
Annette
Resort t

### Access The Movie Database API

In [138]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [140]:
# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 1

# Loop through the titles
for title in titles:
    # Check if we need to sleep before making a request
    if request_counter % 50 == 0:
        print("Sleeping for a moment to respect API limits...")
        time.sleep(1)

    # Add 1 to the request counter
    request_counter += 1
    
    # Perform a "GET" request for The Movie Database
    search_url = f"{url}{title}{tmdb_key_string}"
    search_response = requests.get(search_url).json()


    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    try:
        # Get movie id
        movie_id = search_response['results'][0]['id']

        # Make a request for a the full movie details
        movie_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}"
        movie_response = requests.get(movie_url).json()

        # Execute "GET" request with url
        movie_response = requests.get(movie_url).json()
        
        # Extract the genre names into a list
        genres = [genre['name'] for genre in movie_response.get('genres', [])]

        # Extract the spoken_languages' English name into a list
        spoken_languages = [language['english_name'] for language in movie_response.get('spoken_languages', [])]

        # Extract the production_countries' name into a list
        production_countries = [country['name'] for country in movie_response.get('production_countries', [])]
    
        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        movie_data = {
            'title': movie_response.get('title', ''),
            'original_title': movie_response.get('original_title', ''),
            'budget': movie_response.get('budget', 0),
            'original_language': movie_response.get('original_language', ''),
            'homepage': movie_response.get('homepage', ''),
            'overview': movie_response.get('overview', ''),
            'popularity': movie_response.get('popularity', 0),
            'runtime': movie_response.get('runtime', 0),
            'revenue': movie_response.get('revenue', 0),
            'release_date': movie_response.get('release_date', ''),
            'vote_average': movie_response.get('vote_average', 0),
            'vote_count': movie_response.get('vote_count', 0),
            'genres': genres,
            'spoken_languages': spoken_languages,
            'production_countries': production_countries
        }
        tmdb_movies_list.append(movie_data)
        
        # Print out the title that was found
        print(f"Found {movie_data['title']}")
    except (IndexError, KeyError):
         print(f"{title} not found.")

Found The Attachment Diaries
What’s Love Got to Do With It?’ Probably a Lo not found.
Found You Can Live Forever
Found A Tourist's Guide to Love
Found Other People's Children
Found One True Loves
Found The Lost Weekend: A Love Story
Found A Thousand and One
Found Your Place or Mine
Found Love in the Time of Fentanyl
Found Pamela, A Love Story
Found In from the Side
Found After Love
Found Alcarràs
Found Nelly Knows Mysteries: A Fatal Engagement
Found Lady Chatterley's Lover
Found The Sound of Christmas
Found The Inspection
Found Bones and All
Found My Policeman
Found About Fate
Found Waiting for Bojangles
Found I Love My Dad
Found A Love Song
Found Alone Together
Found Art of Love
Found 2099: The Soldier Protocol
Found Thor: Love and Thunder
Found Both Sides of the Blade
Found Fire of Love
Found Mektoub, My Love
Found Stay Prayed Up
Found Benediction
Found Dinner in America
Found In a New York Minute
Found Anaïs in Love
Found I Love America
Found See You Then
Found La Mami
Found After L

In [142]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
for i in range(min(5, len(tmdb_movies_list))):
    formatted_result = json.dumps(tmdb_movies_list[i], indent=4)
    print(formatted_result)
    print("\n")

{
    "title": "The Attachment Diaries",
    "original_title": "El apego",
    "budget": 0,
    "original_language": "es",
    "homepage": "",
    "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.",
    "popularity": 2.338,
    "runtime": 102,
    "revenue": 0,
    "release_date": "2021-10-07",
    "vote_average": 3.0,
    "vote_count": 4,
    "genres": [
        "Drama",
        "Mystery",
        "Thriller",
        "Horror"
    ],
    "spoken_languages": [
        "Spanish"
    ],
    "production_countries": [
        "Argentina"
    ]
}


{
    "title": "You Can Live Forever",
    "original_title": "You Can Live Forever",
    "budget": 

In [144]:
# Convert the results to a DataFrame
tmdb_df = pd.DataFrame(tmdb_movies_list)
tmdb_df

,title,original_title,budget,original_language,homepage,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genres,spoken_languages,production_countries
0,The Attachment Diaries,El apego,0,es,,"Argentina, 1970s. A desperate young woman goes...",2.338,102,0,2021-10-07,3.000,4,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina]
1,You Can Live Forever,You Can Live Forever,0,en,https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",25.799,96,15055,2023-03-24,6.550,40,"[Drama, Romance]","[English, French]","[Canada, United States of America]"
2,A Tourist's Guide to Love,A Tourist's Guide to Love,0,en,https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",17.863,96,0,2023-04-21,6.303,170,"[Romance, Comedy]","[English, Vietnamese]",[United States of America]
3,Other People's Children,Les Enfants des autres,0,fr,https://www.wildbunch.biz/movie/other-peoples-...,"Rachel loves her life, her students, her frien...",7.963,104,84178,2022-09-21,6.812,194,"[Drama, Comedy]","[French, English]",[France]
4,One True Loves,One True Loves,0,en,,Emma and Jesse are living the perfect life tog...,12.109,100,37820,2023-04-07,6.500,79,"[Romance, Comedy, Drama]","[English, Spanish]","[Czech Republic, United States of America]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,The Last Face,The Last Face,0,en,,"Miguel, a heroic Spanish doctor, puts himself ...",9.985,130,0,2017-01-11,5.500,194,[Drama],"[English, Portuguese]",[United States of America]
150,Lost in Paris,Paris pieds nus,0,fr,,Fiona visits Paris for the first time to assis...,9.218,83,0,2017-01-14,6.000,115,[Comedy],[French],"[Belgium, France]"
151,The Other Half,The Other Half,0,en,http://theotherhalfmovie.com/,A grief-stricken man and a bipolar woman fall ...,3.927,103,0,2016-12-02,6.300,24,"[Drama, Romance]",[English],[Canada]
152,The Ottoman Lieutenant,The Ottoman Lieutenant,40000000,en,https://www.uphe.com/movies/the-ottoman-lieute...,"Lillie, a determined American woman, ventures ...",19.758,111,413844,2017-03-28,6.163,248,"[Romance, Drama, War]","[Turkish, English]","[Turkey, United States of America]"


### Merge and Clean the Data for Export

In [146]:
# Remove any duplicate rows based on the 'title' column in both DataFrames
tmdb_df = tmdb_df.drop_duplicates(subset='title')
df_reviews = df_reviews.drop_duplicates(subset='title')

# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(tmdb_df, df_reviews, on="title", how="inner")

# Adjust Pandas display options to show more columns
pd.set_option('display.max_columns', None)

merged_df

,title,original_title,budget,original_language,homepage,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genres,spoken_languages,production_countries,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,The Attachment Diaries,El apego,0,es,,"Argentina, 1970s. A desperate young woman goes...",2.338,102,0,2021-10-07,3.000,4,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina],https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,You Can Live Forever,You Can Live Forever,0,en,https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",25.799,96,15055,2023-03-24,6.550,40,"[Drama, Romance]","[English, French]","[Canada, United States of America]",https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
2,One True Loves,One True Loves,0,en,,Emma and Jesse are living the perfect life tog...,12.109,100,37820,2023-04-07,6.500,79,"[Romance, Comedy, Drama]","[English, Spanish]","[Czech Republic, United States of America]",https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies;persons: Bracey, Luke (1989- )...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None
3,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,en,https://thelostweekendmovie.com/,May Pang lovingly recounts her life in rock & ...,17.731,95,77145,2023-04-13,6.000,2,[Documentary],[English],[United States of America],https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,subject: Documentary Films and Programs;creati...,2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
4,A Thousand and One,A Thousand and One,0,en,https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,17.021,116,3395595,2023-03-31,6.872,90,"[Drama, Crime]","[English, Spanish, Portuguese]",[United States of America],https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,subject: Movies;creative_works: A Thousand and...,2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,In Search of Fellini,In Search of Fellini,0,en,,"Lucy, a small-town girl from Ohio, discovers t...",4.523,93,0,2017-09-15,5.800,40,"[Drama, Adventure]",[English],[],https://www.nytimes.com/2017/09/14/movies/in-s...,A sheltered young woman enamored of the films ...,The New York Times,subject: Movies;creative_works: In Search of F...,2017-09-14T11:00:24+0000,272,Review: Love of Il Maestro Drives ‘In Search o...,None,None,In Search 

In [148]:
# Remove list brackets and quotation marks on the columns containing lists

# Create a list of the columns that need fixing
columns_to_fix = ['genres', 'spoken_languages', 'production_countries']

# Create a list of characters to remove
characters_to_remove = ["[", "]", "'"]

# Loop through the list of columns to fix
for column in columns_to_fix:
    # Convert the column to type 'str'
    merged_df[column] = merged_df[column].astype(str)
    
    # Loop through characters to remove
    for char in characters_to_remove:
        merged_df[column] = merged_df[column].str.replace(char, "")

# Display the fixed DataFrame
merged_df.head()

,title,original_title,budget,original_language,homepage,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genres,spoken_languages,production_countries,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,The Attachment Diaries,El apego,0,es,,"Argentina, 1970s. A desperate young woman goes...",2.338,102,0,2021-10-07,3.000,4,"Drama, Mystery, Thriller, Horror",Spanish,Argentina,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,You Can Live Forever,You Can Live Forever,0,en,https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",25.799,96,15055,2023-03-24,6.550,40,"Drama, Romance","English, French","Canada, United States of America",https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
2,One True Loves,One True Loves,0,en,,Emma and Jesse are living the perfect life tog...,12.109,100,37820,2023-04-07,6.500,79,"Romance, Comedy, Drama","English, Spanish","Czech Republic, United States of America",https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies;persons: Bracey, Luke (1989- )...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None
3,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,en,https://thelostweekendmovie.com/,May Pang lovingly recounts her life in rock & ...,17.731,95,77145,2023-04-13,6.000,2,Documentary,English,United States of America,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,subject: Documentary Films and Programs;creati...,2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
4,A Thousand and One,A Thousand and One,0,en,https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,17.021,116,3395595,2023-03-31,6.872,90,"Drama, Crime","English, Spanish, Portuguese",United States of America,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,subject: Movies;creative_works: A Thousand and...,2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None


In [150]:
# Drop "byline.person" column
merged_df = merged_df.drop(columns=["byline.person"])

In [152]:
# Delete duplicate rows and reset index
merged_df = merged_df.drop_duplicates(subset='title')
merged_df = merged_df.reset_index(drop=True)
merged_df.head()

,title,original_title,budget,original_language,homepage,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genres,spoken_languages,production_countries,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.organization
0,The Attachment Diaries,El apego,0,es,,"Argentina, 1970s. A desperate young woman goes...",2.338,102,0,2021-10-07,3.000,4,"Drama, Mystery, Thriller, Horror",Spanish,Argentina,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,None
1,You Can Live Forever,You Can Live Forever,0,en,https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",25.799,96,15055,2023-03-24,6.550,40,"Drama, Romance","English, French","Canada, United States of America",https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,None
2,One True Loves,One True Loves,0,en,,Emma and Jesse are living the perfect life tog...,12.109,100,37820,2023-04-07,6.500,79,"Romance, Comedy, Drama","English, Spanish","Czech Republic, United States of America",https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies;persons: Bracey, Luke (1989- )...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,None
3,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,en,https://thelostweekendmovie.com/,May Pang lovingly recounts her life in rock & ...,17.731,95,77145,2023-04-13,6.000,2,Documentary,English,United States of America,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,subject: Documentary Films and Programs;creati...,2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,None
4,A Thousand and One,A Thousand and One,0,en,https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,17.021,116,3395595,2023-03-31,6.872,90,"Drama, Crime","English, Spanish, Portuguese",United States of America,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,subject: Movies;creative_works: A Thousand and...,2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,None


In [154]:
# Export data to CSV without the index
merged_df.to_csv("merged_data.csv", index=False)